In [1]:

import matplotlib.pyplot as plt
import torch
import torchvision
from typing import *
from PIL import Image
from torch import nn
from torchvision import transforms
device="cuda" if torch.cuda.is_available() else "cpu"
class_names=['Apple___Apple_scab',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  'Apple___healthy',
  'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
  'Corn_(maize)___Common_rust_',
  'Corn_(maize)___Northern_Leaf_Blight',
  'Corn_(maize)___healthy',
  'Potato___Bacteria',
  'Potato___Early_blight',
  'Potato___Late_blight',
  'Potato___Phytopthora',
  'Potato___Virus',
  'Potato___healthy',
  'Tomato___Bacterial_spot',
  'Tomato___Early_blight',
  'Tomato___Late_blight',
  'Tomato___Leaf_Mold',
  'Tomato___Septoria_leaf_spot',
  'Tomato___Spider_mites Two-spotted_spider_mite',
  'Tomato___Target_Spot',
  'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
  'Tomato___Tomato_mosaic_virus',
  'Tomato___healthy']
weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT 
model=torchvision.models.efficientnet_b0(weights=weights)

for param in model.features.parameters():
    param.requires_grad = False

# ! changing input and output layers (classsifier)
model.classifier=nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names), bias=True).to(device)
)
model.load_state_dict(torch.load("model_efficientnetb0_epoch_25.pth"))
def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str, 
                        class_names: List[str],
                        image_size: Tuple[int, int] = (224, 224),
                        transform: torchvision.transforms = None,
                        device: torch.device=device):
    
    
    # 2. Open image
    img = Image.open(image_path)

    # 3. Create transformation for image (if one doesn't exist)
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

    ### Predict on image ### 

    # 4. Make sure the model is on the target device
    model.to(device)

    # 5. Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
      # 6. Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
      transformed_image = image_transform(img).unsqueeze(dim=0)

      # 7. Make a prediction on image with an extra dimension and send it to the target device
      target_image_pred = model(transformed_image.to(device))

    # 8. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # 9. Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # 10. Plot image with predicted label and probability 
    plt.figure()
    plt.imshow(img)
    plt.title(f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}")
    plt.axis(False);
custom_image_path="train\Apple___Black_rot\Copy of d56311b8-ffb7-4f58-b521-c8be2a31f980___JR_FrgE.S 2799_new30degFlipLR.JPG"
pred_and_plot_image(model=model,
                    image_path=custom_image_path,
                    class_names=class_names)

<>:92: SyntaxWarning: invalid escape sequence '\A'
<>:92: SyntaxWarning: invalid escape sequence '\A'
C:\Users\ArnavBhatia\AppData\Local\Temp\ipykernel_33144\3363485123.py:92: SyntaxWarning: invalid escape sequence '\A'
  custom_image_path="train\Apple___Black_rot\Copy of d56311b8-ffb7-4f58-b521-c8be2a31f980___JR_FrgE.S 2799_new30degFlipLR.JPG"
C:\Users\ArnavBhatia\AppData\Local\Temp\ipykernel_33144\3363485123.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless th

FileNotFoundError: [Errno 2] No such file or directory: 'model_efficientnetb0_epoch_25.pth'